In [ ]:
from yahoofinancials import YahooFinancials
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import datetime

In [ ]:
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and (not y%100==0 or y%400 == 0) else 28,
        31,30,31,30,31,31,30,31,30,31][m-1])
    return date.replace(day=d,month=m, year=y)

def support_resistance_line(high_frame, low_frame, current):
    high, low = max(high_frame), min(low_frame)
    P = (high + low + current)/3
    R1 = (P*2)-low
    R2 = P+(high-low)
    S1=(P*2)-high
    S2 = P-(high-low)
    return max(S1,S2), min(R1,R2)

def moving_average(column, day=200):
    return column.rolling(day).mean()

In [ ]:
now = datetime.datetime.now().date()
months_before = str(monthdelta(now, -18))
ticker = "MSFT"
yf = YahooFinancials(ticker)

current_price = yf.get_stock_price_data()[ticker]["regularMarketPrice"]

history = yf.get_historical_price_data(months_before, str(now),
                                       "daily")[ticker]["prices"]

In [ ]:
from_history = lambda string : [history[i][str(string)] for i in range(len(history))]

df = pd.DataFrame({"Date" : from_history("formatted_date"),
                   "Open" : from_history("open"),
                   "Close" : from_history("close"),
                   "High" : from_history("high"),
                   "Low" : from_history("low")})
df["Day Average"] = [(df.High[i] + df.Low[i])/2 for i in range(df.shape[0])]
print(df)

In [ ]:
six_months = -180
x = df.Date
ma_y = moving_average(df.Close)
s_y, r_y = support_resistance_line(df.High[six_months:], df.Low[six_months:], current_price)
supp_y = [s_y for _ in range(len(x[six_months:]))]

fig = go.Figure(data=[go.Candlestick(
                            x=x,
                            name=ticker,
                            open=df.Open,
                            high=df.High,
                            low=df.Low,
                            close=df.Close)])

fig.add_trace(go.Scatter(x=x, y=ma_y, 
                        mode='lines', marker = dict( color = '#E377C2'),
                        name='Moving average' ))

fig.add_trace(go.Scatter(x=x[six_months:], y=supp_y, 
                        mode='lines', marker = dict( color = 'blue'),
                        name='Support line' ))

fig.show()

In [ ]:
"""
Generall algorithm:
1. check if price near supp line
2. check if above 200 MA
3. check if buyers in control
"""

In [ ]:
def hammer():
    if df.Close > df.Open:
        return True

def above_MA():
    if df.Open > ma_y:
        return True

def close_to_support():
    if s_y * 1.05 < df.Open < s_y * 0.95:
        return True

df.iloc[(hammer()) & (above_MA()) & (close_to_support())]